# 무신사 댓글분석

## 댓글 불러오기
한번에 500페이지씩 불러오고 별점 1,2점이면 0 10점이면 1을 부여한상태로 저장 대략 30분정도 걸림

In [136]:
import urllib.request
import bs4
s_num = int(input())
e_num = int(input())
now = datetime.datetime.now()
print(now)
review = []
for i in range(s_num,e_num+1):
    url = "https://store.musinsa.com/app/reviews/lists?type=goods&year_date=2020&month_date=3&day_date=&max_rt=2020&min_rt=2018&brand=&page="+str(i)+"&sort=new&enc_val=&tag=&s_type=all&q="
    html = urllib.request.urlopen(url)
    bs_obj = bs4.BeautifulSoup(html,"html.parser")
    div = bs_obj.findAll("div",{"class":"postContents container est_cont"})
    target = ['2','4','10']
    for i in div:
        div = i.find("div",{"class":"n-score"})
        if div.find("span")["class"][1][5:] in target:
            empty = []
            span = i.find("span",{"class":"content-review"})
            empty.append(span.text[14:].strip('\n').strip('\t'))
            if div.find("span")["class"][1][5:] == '10':
                empty.append(1)
            else:
                empty.append(0)
            review.append(empty)
        
later = datetime.datetime.now()

print(later)
print(review)

4501
5000
2020-05-15 17:43:27.425109


URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

## 표본 추출
부정 리뷰수가 압도적으로 적기때문에 불러온 댓글 중 부정댓글은 전부 사용하고 긍정댓글은 너무 많기에 부정댓글 수만큼만 임의추출

In [135]:
import csv
import re
neg = []
pos = []

f = open('review_final.csv','a',encoding='utf-8-sig')
wr = csv.writer(f)
for i in review:
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' #한글 자모음 제거
    text = re.sub(pattern=pattern, repl=' ', string=i[0])
    pattern = '[^\w\s]' #특수기호 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    
for i in review:
    if i[1] == 0:
        neg.append(i)
    else:
        pos.append(i)
pos_choice = random.sample(pos,len(neg))

for i in neg:
    wr.writerow(i)

for i in pos_choice:
    wr.writerow(i)
    
f.close()
print(len(neg))

78


## 저장내용 불러오기

In [137]:
import csv
new_t = []
f = open('review_final.csv','r',encoding='utf-8-sig')
rdf = csv.reader(f)

for i in rdf:
    new_t.append(i)

len(new_t)

1818

## 단어 토큰화

In [139]:
from konlpy.tag import Okt
import nltk
import re


okt = Okt()

def modify(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' #한글 자모음 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '[^\w\s]' #특수기호 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    return text

def tokenize(doc):
    return ['/'.join(t) for t in okt.pos(doc,norm=True,stem=True)]



training = [(tokenize(modify(row[0])),row[1]) for row in new_t]#[('단어/형태',...,0 or 1)]

print()
token = [t for d in training for t in d[0]] #단어들만 따로 모음
text = nltk.Text(token)

## 최빈값 추출 및 단어 빈도 수 저장

In [140]:
selected_words = [f[0] for f in text.vocab().most_common(3002)] #more랑 \n때문에 두개 더 추가해야함
selected_words.remove('\n/Foreign')
selected_words.remove('more/Alpha')
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]



## training setting

In [141]:
train_x = [term_frequency(d) for d,_ in training]

#test_x = [term_frequency(d) for d,_in test]

train_y = [c for _,c in training]

#test_y = [c for _,c in test]

In [142]:
import numpy as np
x_train = np.asarray(train_x).astype('float32')
#x_test = np.asarray(test_x).astype('float32')
y_train = np.asarray(train_y).astype('float32')
#y_test = np.asarray(test_y).astype('float32')

## 학습

In [143]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64,activation='relu',input_shape=(3000,)))#실제 쓸 단어갯수
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),loss = losses.binary_crossentropy,metrics=[metrics.binary_accuracy])

model.fit(x_train,y_train,epochs=150,batch_size=512)



/Users/leesangyun/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/leesangyun/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/leesangyun/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/leesangyun/opt/anaconda3/lib/python3.7/site-pack

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/150
1818/1818 [==============================] - 1s 368us/sample - loss: 0.6735 - binary_accuracy: 0.6870
Epoch 2/150
1818/1818 [==============================] - 0s 21us/sample - loss: 0.5756 - binary_accuracy: 0.8911
Epoch 3/150
1818/1818 [==============================] - 0s 25us/sample - loss: 0.4537 - binary_accuracy: 0.9274
Epoch 4/150
1818/1818 [==============================] - 0s 18us/sample - loss: 0.3436 - binary_accuracy: 0.9444
Epoch 5/150
1818/1818 [==============================] - 0s 117us/sample - loss: 0.2634 - binary_accuracy: 0.9560
Epoch 6/150
1818/1818 [==============================] - 0s 19us/sample - loss: 0.2065 - binary_accuracy: 0.9637
Epoch 7/150
1818/1818 [==============================] - 0s 19us/sample - loss: 0.1649 - binary_accuracy: 0.9714
Epoch 8/150
1818/1818 [==============================] - 0s 20us/sample - loss: 0.1342 

1818/1818 [==============================] - 0s 18us/sample - loss: 1.0627e-06 - binary_accuracy: 1.0000
Epoch 67/150
1818/1818 [==============================] - 0s 18us/sample - loss: 9.2388e-07 - binary_accuracy: 1.0000
Epoch 68/150
1818/1818 [==============================] - 0s 18us/sample - loss: 7.9859e-07 - binary_accuracy: 1.0000
Epoch 69/150
1818/1818 [==============================] - 0s 19us/sample - loss: 7.0138e-07 - binary_accuracy: 1.0000
Epoch 70/150
1818/1818 [==============================] - 0s 19us/sample - loss: 6.1474e-07 - binary_accuracy: 1.0000
Epoch 71/150
1818/1818 [==============================] - 0s 18us/sample - loss: 5.4895e-07 - binary_accuracy: 1.0000
Epoch 72/150
1818/1818 [==============================] - 0s 18us/sample - loss: 4.7726e-07 - binary_accuracy: 1.0000
Epoch 73/150
1818/1818 [==============================] - 0s 19us/sample - loss: 4.3946e-07 - binary_accuracy: 1.0000
Epoch 74/150
1818/1818 [==============================] - 0s 18us/sam

1818/1818 [==============================] - 0s 22us/sample - loss: 1.1373e-07 - binary_accuracy: 1.0000
Epoch 136/150
1818/1818 [==============================] - 0s 20us/sample - loss: 1.1396e-07 - binary_accuracy: 1.0000
Epoch 137/150
1818/1818 [==============================] - 0s 20us/sample - loss: 1.1326e-07 - binary_accuracy: 1.0000
Epoch 138/150
1818/1818 [==============================] - 0s 20us/sample - loss: 1.1379e-07 - binary_accuracy: 1.0000
Epoch 139/150
1818/1818 [==============================] - 0s 20us/sample - loss: 1.1345e-07 - binary_accuracy: 1.0000
Epoch 140/150
1818/1818 [==============================] - 0s 20us/sample - loss: 1.1335e-07 - binary_accuracy: 1.0000
Epoch 141/150
1818/1818 [==============================] - 0s 20us/sample - loss: 1.1351e-07 - binary_accuracy: 1.0000
Epoch 142/150
1818/1818 [==============================] - 0s 21us/sample - loss: 1.1336e-07 - binary_accuracy: 1.0000
Epoch 143/150
1818/1818 [==============================] - 0s 

In [144]:
def predict(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'),axis=0)
    score = float(model.predict(data))
    print(score)
    if(score>0.5):
        print("[{}]는 {:.2f}%확률로 만족 리뷰입니다.".format(review,score*100))
    else:
        print("[{}]는 {:.2f}%확률로 불만족 리뷰입니다.".format(review,(1-score)*100))

## Predict

In [145]:
predict("좋습니다 정말 만족합니다")
predict("별로입니다 다시는 사기 싫습니다")
predict("기대 많이 했는데 생각만큼 편하지는 않네요 그냥 그래요")
predict("그냥 무난해요 좋지도 않고 나쁘지도 않고")
predict("많이들 입기는하던데 솔직히 모르겠음 ")

0.9999996423721313
[좋습니다 정말 만족합니다]는 100.00%확률로 만족 리뷰입니다.
4.983027793059591e-06
[별로입니다 다시는 사기 싫습니다]는 100.00%확률로 불만족 리뷰입니다.
0.44577816128730774
[기대 많이 했는데 생각만큼 편하지는 않네요 그냥 그래요]는 55.42%확률로 불만족 리뷰입니다.
0.9999053478240967
[그냥 무난해요 좋지도 않고 나쁘지도 않고]는 99.99%확률로 만족 리뷰입니다.
0.0025326251052320004
[많이들 입기는하던데 솔직히 모르겠음 ]는 99.75%확률로 불만족 리뷰입니다.
